# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6decc98c70>
├── 'a' --> tensor([[-1.5502, -0.5344,  1.0017],
│                   [ 0.3730, -0.5865,  0.5163]])
└── 'x' --> <FastTreeValue 0x7f6decc98c40>
    └── 'c' --> tensor([[-0.5064,  1.3113,  0.7474, -1.5178],
                        [ 0.3662,  0.6748, -0.8927,  0.9459],
                        [ 0.6007, -1.6776,  1.7443,  0.0511]])

In [4]:
t.a

tensor([[-1.5502, -0.5344,  1.0017],
        [ 0.3730, -0.5865,  0.5163]])

In [5]:
%timeit t.a

80 ns ± 0.591 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6decc98c70>
├── 'a' --> tensor([[ 1.6545,  0.4938, -0.5527],
│                   [ 0.6884, -1.1699, -0.3904]])
└── 'x' --> <FastTreeValue 0x7f6decc98c40>
    └── 'c' --> tensor([[-0.5064,  1.3113,  0.7474, -1.5178],
                        [ 0.3662,  0.6748, -0.8927,  0.9459],
                        [ 0.6007, -1.6776,  1.7443,  0.0511]])

In [7]:
%timeit t.a = new_value

94.1 ns ± 0.833 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5502, -0.5344,  1.0017],
               [ 0.3730, -0.5865,  0.5163]]),
    x: Batch(
           c: tensor([[-0.5064,  1.3113,  0.7474, -1.5178],
                      [ 0.3662,  0.6748, -0.8927,  0.9459],
                      [ 0.6007, -1.6776,  1.7443,  0.0511]]),
       ),
)

In [10]:
b.a

tensor([[-1.5502, -0.5344,  1.0017],
        [ 0.3730, -0.5865,  0.5163]])

In [11]:
%timeit b.a

73.9 ns ± 0.804 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7552, -1.4359, -1.9564],
               [-1.3188,  0.7415, -0.0029]]),
    x: Batch(
           c: tensor([[-0.5064,  1.3113,  0.7474, -1.5178],
                      [ 0.3662,  0.6748, -0.8927,  0.9459],
                      [ 0.6007, -1.6776,  1.7443,  0.0511]]),
       ),
)

In [13]:
%timeit b.a = new_value

660 ns ± 4.81 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.08 µs ± 19.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.1 µs ± 307 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

304 µs ± 29.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

262 µs ± 9.64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6decc98850>
├── 'a' --> tensor([[[-1.5502, -0.5344,  1.0017],
│                    [ 0.3730, -0.5865,  0.5163]],
│           
│                   [[-1.5502, -0.5344,  1.0017],
│                    [ 0.3730, -0.5865,  0.5163]],
│           
│                   [[-1.5502, -0.5344,  1.0017],
│                    [ 0.3730, -0.5865,  0.5163]],
│           
│                   [[-1.5502, -0.5344,  1.0017],
│                    [ 0.3730, -0.5865,  0.5163]],
│           
│                   [[-1.5502, -0.5344,  1.0017],
│                    [ 0.3730, -0.5865,  0.5163]],
│           
│                   [[-1.5502, -0.5344,  1.0017],
│                    [ 0.3730, -0.5865,  0.5163]],
│           
│                   [[-1.5502, -0.5344,  1.0017],
│                    [ 0.3730, -0.5865,  0.5163]],
│           
│                   [[-1.5502, -0.5344,  1.0017],
│                    [ 0.3730, -0.5865,  0.5163]]])
└── 'x' --> <FastTreeValue 0x7f6d37fb4eb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.4 µs ± 839 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6dc41c1fa0>
├── 'a' --> tensor([[-1.5502, -0.5344,  1.0017],
│                   [ 0.3730, -0.5865,  0.5163],
│                   [-1.5502, -0.5344,  1.0017],
│                   [ 0.3730, -0.5865,  0.5163],
│                   [-1.5502, -0.5344,  1.0017],
│                   [ 0.3730, -0.5865,  0.5163],
│                   [-1.5502, -0.5344,  1.0017],
│                   [ 0.3730, -0.5865,  0.5163],
│                   [-1.5502, -0.5344,  1.0017],
│                   [ 0.3730, -0.5865,  0.5163],
│                   [-1.5502, -0.5344,  1.0017],
│                   [ 0.3730, -0.5865,  0.5163],
│                   [-1.5502, -0.5344,  1.0017],
│                   [ 0.3730, -0.5865,  0.5163],
│                   [-1.5502, -0.5344,  1.0017],
│                   [ 0.3730, -0.5865,  0.5163]])
└── 'x' --> <FastTreeValue 0x7f6decc92220>
    └── 'c' --> tensor([[-0.5064,  1.3113,  0.7474, -1.5178],
                        [ 0.3662,  0.6748, -0.8927,  0.9459],
                 

In [23]:
%timeit t_cat(trees)

45 µs ± 817 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

82.4 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.5064,  1.3113,  0.7474, -1.5178],
                       [ 0.3662,  0.6748, -0.8927,  0.9459],
                       [ 0.6007, -1.6776,  1.7443,  0.0511]],
              
                      [[-0.5064,  1.3113,  0.7474, -1.5178],
                       [ 0.3662,  0.6748, -0.8927,  0.9459],
                       [ 0.6007, -1.6776,  1.7443,  0.0511]],
              
                      [[-0.5064,  1.3113,  0.7474, -1.5178],
                       [ 0.3662,  0.6748, -0.8927,  0.9459],
                       [ 0.6007, -1.6776,  1.7443,  0.0511]],
              
                      [[-0.5064,  1.3113,  0.7474, -1.5178],
                       [ 0.3662,  0.6748, -0.8927,  0.9459],
                       [ 0.6007, -1.6776,  1.7443,  0.0511]],
              
                      [[-0.5064,  1.3113,  0.7474, -1.5178],
                       [ 0.3662,  0.6748, -0.8927,  0.9459],
                       [ 0.6007, -1.6776,  1.7443,  0.0511]],

In [26]:
%timeit Batch.stack(batches)

110 µs ± 3.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.5064,  1.3113,  0.7474, -1.5178],
                      [ 0.3662,  0.6748, -0.8927,  0.9459],
                      [ 0.6007, -1.6776,  1.7443,  0.0511],
                      [-0.5064,  1.3113,  0.7474, -1.5178],
                      [ 0.3662,  0.6748, -0.8927,  0.9459],
                      [ 0.6007, -1.6776,  1.7443,  0.0511],
                      [-0.5064,  1.3113,  0.7474, -1.5178],
                      [ 0.3662,  0.6748, -0.8927,  0.9459],
                      [ 0.6007, -1.6776,  1.7443,  0.0511],
                      [-0.5064,  1.3113,  0.7474, -1.5178],
                      [ 0.3662,  0.6748, -0.8927,  0.9459],
                      [ 0.6007, -1.6776,  1.7443,  0.0511],
                      [-0.5064,  1.3113,  0.7474, -1.5178],
                      [ 0.3662,  0.6748, -0.8927,  0.9459],
                      [ 0.6007, -1.6776,  1.7443,  0.0511],
                      [-0.5064,  1.3113,  0.7474, -1.5178],
                   

In [28]:
%timeit Batch.cat(batches)

190 µs ± 4.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

586 µs ± 9.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
